In [376]:
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import accuracy_score



### Criar base

Essa função é usada para criar a base de treino do algoritmo. Assim ela é usada para definir as saidas lógicas baseado no tipo de gate escolhido. Se for "AND", a função testa de todos os valores são iguaus a 1, uma vez que a porta AND só retorna verdadeiro se todas as entradas forem verdadeiras. Se for "OR", a função testa se algum valor é 1, uma vez que a porta or retorna verdadeiro se alguma entrada for verdadeira. Se for "XOR", a função usa da função logical_xor do numpy para fazer a lógica xor. A função faz essa base com base nos inputs do tamanho determinado, onde cada linha representa uma combinação única de valores booleanos (0 e 1) para um conjunto de entradas.

In [377]:
def create_dataset(num_inputs, logic_gate):
    inputs = np.array(list(np.ndindex((2,) * num_inputs)))

    if logic_gate.lower() == 'and':
        outputs = np.all(inputs, axis=1).astype(int)
    elif logic_gate.lower() == 'or':
        outputs = np.any(inputs, axis=1).astype(int)
    elif logic_gate.lower() == 'xor':
        #print("1º ->",inputs[:, 0])
        #print("2º ->",inputs[:, 1])

        # XOR é definido como 1 apenas quando o número de 1s nas entradas é ímpar
        outputs = np.logical_xor(inputs[:, 0], inputs[:, 1]).astype(int)
    else:
        raise ValueError("Gate must be 'AND', 'OR', or 'XOR'.")

    return inputs, outputs


In [378]:

def main():
    # Solicita ao usuário o número de entradas e o tipo de porta lógica
    num_inputs = int(input("Digite o número de entradas (por exemplo, 2 ou 10): "))
    gate_type = input("Digite o tipo de porta lógica (AND, OR, ou XOR): ")

    # Cria o conjunto de dados com base nas entradas do usuário
    x, y = create_dataset(num_inputs, gate_type)
    print("Base\n - entradas\n",x," - saidas",y, "\n")

    # Inicializa um classificador MLP (Multilayer Perceptron)
    mlpR = MLPClassifier(activation="relu", learning_rate="adaptive")
    mlpT = MLPClassifier(activation="tanh", learning_rate="adaptive")
    mlpS = MLPClassifier(activation="logistic", learning_rate="adaptive")

    # Treina o MLP usando os dados de entrada e saída
    mlpR.fit(x, y)
    mlpT.fit(x, y)
    mlpS.fit(x, y)

    # Testa o MLP com novas entradas fornecidas pelo usuário
    while True:
        user_inputs = [int(input(f"Digite a entrada {i + 1} (0 ou 1): ")) for i in range(num_inputs)]
        
        # Faz uma previsão com o MLP treinado
        predictionR = mlpR.predict([user_inputs])
        predictionT = mlpT.predict([user_inputs])
        predictionS = mlpS.predict([user_inputs])
        
        # Exibe o resultado da previsão
        print(f"Resultado:\n - Relu:{predictionR}\n - Tahn:{predictionT}\n - Sigmore:{predictionS}")
        
        predictionRa = mlpR.predict(x)
        predictionTa = mlpT.predict(x)
        predictionSa = mlpS.predict(x)
        
        acr = accuracy_score(y, predictionRa)
        act = accuracy_score(y, predictionTa)
        acs = accuracy_score(y, predictionSa)
        print(f"\n accuracy\n - Relu:{acr}\n - Tahn:{act}\n - Sigmore:{acs}")
        
        


## Análises
### 1. Taxa de Aprendizado (`learning_rate`):

A taxa de aprendizado é um hiperparâmetro crucial em algoritmos de otimização, como o usado no treinamento de redes neurais. Ela controla o tamanho dos passos que o otimizador dá ao ajustar os pesos durante o treinamento. A escolha adequada da taxa de aprendizado é essencial:

- **Alta Taxa de Aprendizado:** Pode resultar em convergência mais rápida, mas também pode levar a oscilações e a overshooting, onde o algoritmo pode nunca convergir.
- **Baixa Taxa de Aprendizado:** Pode garantir convergência, mas o treinamento pode ser muito lento.

**Tipos**

- **‘constant’**  - é constante e dado por *learning_rate_init* (default= 0.001)
- **‘invscaling’** - diminui gradualmente o *learning rate* usando um expoente inverso a t: effective_learning_rate = learning_rate_init / pow(t, power_t)
- **‘adaptive’** - mantém o *learning rate* constante contanto que a perda de treinamento continue redunzindo

### 2. Termo de Bias (`bias`):

O termo de bias é um parâmetro adicional em cada camada da rede neural. Ele é adicionado aos produtos escalares dos pesos e entradas antes de passar pela função de ativação. O bias permite à rede neural aprender uma transformação mais flexível e ajustar o modelo para se adaptar melhor aos dados.

### 3. Funções de Ativação:

Funções de ativação introduzem não linearidades na rede neural e são essenciais para permitir que a rede aprenda relações complexas nos dados. Duas funções comuns são a **Função Sigmoide** e a **Função Tangente Hiperbólica (tanh)**

In [379]:
main()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Base
 - entradas
 [[0 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 1]
 [1 0 0]
 [1 0 1]
 [1 1 0]
 [1 1 1]]  - saidas [0 0 0 0 0 0 0 1] 

Resultado:
 - Relu:[1]
 - Tahn:[1]
 - Sigmore:[0]
accuracy
 - Relu:1.0
 - Tahn:1.0
 - Sigmore:0.875


ValueError: invalid literal for int() with base 10: ''

### Resultados
- Com base nos experimentos foi possivel perceber que a função de ativação de Tahn levou a resultados mais exatos em até 3 entradas.
- a partir de 4 entradas, resultados mais raros - como 1 na porta And - começaram a sererem previstos errados